# Using spatial autocorrelation on CORONAVIRUS

In [3]:
!pip install --user arcgis
!pip install --user libpysal
!pip install --user esda

You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [9]:
from datetime import date, timedelta
import matplotlib.pyplot as plt
import pandas
import urllib.request

class hopkins_timeseries:
    def __init__(self, states, regions, timeseries):
        self.__states = states
        self.__regions = regions
        self.__timeseries = timeseries
    
    def get(self):
        return self.__timeseries
    
    def plot(self, names, last_days=60):
        height = 2 * len(names)
        if 0 < height:
            end_index = date.today()
            start_index = end_index-timedelta(days=last_days)
            self.__timeseries[start_index:end_index][names].plot(subplots=True, figsize=(15, height))
            
    def plot_all(self, data, last_days=60):
        height = 2 * len(data.columns)
        if 0 < height:
            end_index = date.today()
            start_index = end_index-timedelta(days=last_days)
            data.loc[start_index:end_index].plot(subplots=True, figsize=(15, height))
    
    def plot_regions(self, names, last_days=60):
        aggregated_regions = self.__aggregate_regions_by_sum(names)
        self.plot_all(aggregated_regions, last_days)
    
    def plot_states(self, names, last_days=60):
        states = self.__get_states(names)
        self.plot(states, last_days)
        
    def __aggregate_regions_by_sum(self, names):
        aggregated_timeseries = pandas.DataFrame()
        for name in names:
            states_from_region = self.__get_states_from_region(name)
            if 0 < len(states_from_region):
                aggregated_timeseries[name] = self.__timeseries[states_from_region].sum(axis=1)                
            elif name in self.__timeseries.columns:
                aggregated_timeseries[name] = self.__timeseries[name]
        return aggregated_timeseries
        
    def __get_states_from_region(self, name):
        states = []
        region_indices = [index for index in range(0, len(self.__regions)) if name == self.__regions[index]]
        for region_index in region_indices:
            state = self.__states[region_index]
            if pandas.notna(state):
                states.append(state)
        return states
    
    def __get_states(self, names):
        states = []
        for name in names:
            states_from_region = self.__get_states_from_region(name)
            if 0 < len(states_from_region):
                states += states_from_region
            else:
                state_indices = [index for index in range(0, len(self.__states)) if name == self.__states[index]]
                for state_index in state_indices:
                    state = self.__states[state_index]
                    if pandas.notna(state):
                        states.append(state)
        return states
        

def query_hopkins_data():
    url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv"
    with urllib.request.urlopen(url) as stream:
        timeseries_hopkins = pandas.read_csv(stream)
        return timeseries_hopkins
        
def query_hopkins_timeseries():
    url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv"
    with urllib.request.urlopen(url) as stream:
        timeseries_hopkins = pandas.read_csv(stream)
        date_columns = timeseries_hopkins.columns[4:]
        timeseries_indexed = timeseries_hopkins[date_columns].T
        timeseries_indexed.index = pandas.to_datetime(timeseries_indexed.index)
        states = timeseries_hopkins["Province/State"]
        regions = timeseries_hopkins["Country/Region"]
        timeseries_indexed.columns = [states[index] if 0 < pandas.notna(states[index]) else regions[index] for index in range(0, len(states))]
        return hopkins_timeseries(states, regions, timeseries_indexed)

In [10]:
corona_data = query_hopkins_data()
corona_data

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,82,114,147,177,212,272,322,411,599,599.0
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,773,839,825,878,889,924,963,1007,1086,1086.0
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,212,226,243,266,313,345,385,432,455,455.0
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,2,2.0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,238,428,566,673,790,900,1030,1183,1306,1306.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,NaN,Jersey,49.1900,-2.1100,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
497,NaN,Puerto Rico,18.2000,-66.5000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
498,NaN,Republic of the Congo,-1.4400,15.5560,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
499,NaN,The Bahamas,24.2500,-76.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
